In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## MultiInput Convolutional XRay Classification for COVID-19

## Introducion

The COVID-19 pandemic remains to have a destructive effect on the health and well-being of the global population.  A vital step in the battle against COVID-19 is the successful screening of infected patients, together with one of the significant screening methods being radiology examination using chest radiography.   Motivated by this and motivated by the open-source efforts of the research area, in this research, we introduce a Multi-Input Transfer Learning COVID-Net fuzzy convolutional neural network to the detection of COVID-19 instances from torso X-ray.   Furthermore, we investigate several Convolutional Networks  COVID-Net forecasts with an explainability method in an attempt not only to gain deeper insights into critical factors associated with COVID instances, which can aid clinicians in enhanced screening.                                                      

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def install(api):
    import sys
    import subprocess
    import pkg_resources

    required = {api}
    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = required - installed
    if missing:
        !pip install -U $api

In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
import dask.dataframe as dd



In [ ]:
from IPython.display import Image

Image("/kaggle/input/papercovidmethod/Processo.png")


In [ ]:
## Read train images

train_images=[]

import glob, os
os.chdir("/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/")
for file in glob.glob("*.jpeg"):
    train_images.append(file)

In [ ]:
## Read test images

test_images=[]

import glob, os
os.chdir("/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/")
for file in glob.glob("*.jpeg"):
    test_images.append(file)

    
test_images2=[]
import glob, os
os.chdir("/kaggle/input/covid-19-x-ray-10000-images/dataset/covid")
for file in glob.glob("*.jpeg"):
    test_images2.append(file)
    
test_images_normal=[]
import glob, os
os.chdir("/kaggle/input/covid-19-x-ray-10000-images/dataset/normal")
for file in glob.glob("*.jpeg"):
    test_images_normal.append(file)
    


In [ ]:
print('Length of Covid Dataset 2 ',len(test_images2))
print('Length of Covid Dataset 2 ',len(test_images_normal))

In [ ]:
install('scikit-fuzzy')

import skfuzzy as fuzz

In [ ]:
#Read dataset metadataa
import pandas as pd
x_ray=pd.read_csv('/kaggle/input/coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv')

x_ray

In [ ]:
x_ray.groupby(['Label']).count()

In [ ]:
x_ray[x_ray['Label_2_Virus_category'].isnull()].groupby(['Label']).count()

In [ ]:
x_ray.groupby(['Label_2_Virus_category']).count()

In [ ]:

x_ray_covid=x_ray[(x_ray['Label_2_Virus_category']=='COVID-19') | (x_ray['Label_2_Virus_category']=='SARS')]
x_ray_normal=x_ray[x_ray['Label']=='Normal']

In [ ]:
x_ray_covid.groupby(['Label_2_Virus_category']).count()

In [ ]:
import matplotlib.pyplot as plt
x_ray_covid['Label_2_Virus_category'].hist()
plt.show()

In [ ]:

x_ray=pd.concat([x_ray_covid,x_ray_normal])
x_ray=x_ray.sample(frac=1)
x_ray=x_ray.drop(['Label_2_Virus_category','Label_1_Virus_category'],axis=1)

### COVID-19 Dataset

In [ ]:
x_ray_covid

### Dataset Metadata 

In [ ]:
x_ray

In [ ]:
df_train_=x_ray[x_ray['Dataset_type']=='TRAIN']
df_test_=x_ray[x_ray['Dataset_type']=='TEST']

In [ ]:
df_train_['filename']=df_train_['X_ray_image_name'].apply(lambda x:'/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'+x)
df_test_['filename']=df_test_['X_ray_image_name'].apply(lambda x:'/kaggle/input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'+x)

In [ ]:
import pandas as pd
df_train=df_train_
df_test=df_test_


In [ ]:
df_train.head(5)
df_train.loc[:,'class']=df_train['Label']
df_test.loc[:,'class']=df_test['Label']

In [ ]:
df_train_benign=df_train[df_train['Label']=='Normal']
df_train_malign=df_train[df_train['Label']!='Normal']
df_train_benign=df_train_benign[:len(df_train_malign)]
print('benign ',len(df_train_benign))
print('malign ',len(df_train_malign))
df_train_balanced=pd.concat([df_train_benign,df_train_malign],axis=0)
df_train_balanced=df_train_balanced.sample(frac=1)
df_train_balanced['class']=df_train_balanced['Label']

In [ ]:
df_test_benign=df_test[df_test['Label']=='Normal'][0:36]
df_test_malign=df_test[df_test['Label']!='Normal'][0:36]
df_test_benign=df_test_benign[:len(df_test_benign)]
print('benign ',len(df_test_benign))
print('malign ',len(df_test_malign))
df_test_balanced=pd.concat([df_test_benign,df_test_malign],axis=0)
df_test_balanced=df_test_balanced.sample(frac=1)
df_test_balanced['class']=df_test_balanced['Label']

In [ ]:
len(test_images2)

In [ ]:
df_test_balanced=df_test_balanced.filter(['filename','class'])
for f in test_images2:
    df_test_balanced=df_test_balanced.append({'filename':'/kaggle/input/covid-19-x-ray-10000-images/dataset/covid/'+f,'class':'Pneumonia'},ignore_index=True)
df_test_balanced=df_test_balanced.sample(frac=1)  

In [ ]:
df_test_balanced

In [ ]:
from IPython.display import Image

Image("/kaggle/input/papercovidmethod/Processo (2).png")


### Balancing dataset samples

In [ ]:
print('malign ',len(df_test_balanced[df_test_balanced['class']=='Pneumonia']))
print('benign ',len(df_test_balanced[df_test_balanced['class']=='Normal']))

In [ ]:

df_valid_balanced=df_test_balanced

In [ ]:
len(df_valid_balanced)

## Reading data with Image Data Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen_no_fuzzy=ImageDataGenerator(rescale=1/255)
train_no_fuzzy=gen_no_fuzzy.flow_from_dataframe(df_train_balanced,batch_size=124,shuffle=False)
valid_no_fuzzy=gen_no_fuzzy.flow_from_dataframe(df_valid_balanced,batch_size=72,shuffle=False)
test_no_fuzzy=gen_no_fuzzy.flow_from_dataframe(df_test_balanced,batch_size=72,shuffle=False)

### Dataset images

In [ ]:
from skimage import transform,io
import cv2

def fuzzy(image,a,b,c,d):
    
    
    
    mfx = fuzz.trapmf(image.flatten(),  [0.4, 0.6,200,200])
    
 
    
    return mfx.reshape(256,256,3)




import skimage.segmentation as seg
import skimage.color as color

def seg_(image):
    
    
    
    image_aux= seg.slic(image,n_segments=3)
    img2 = np.zeros_like(image)
    img2[:,:,0]=image_aux
    img2[:,:,1]=image_aux
    img2[:,:,2]=image_aux
    
    return  img2

    


    

In [ ]:
import cv2

train_images,labels=train_no_fuzzy.next()
filenames=train_no_fuzzy.filenames
valid_images,labels_valid=valid_no_fuzzy.next()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
for i in range(9):
    
    plt.subplot(3, 3, i + 1,aspect = 'equal')
    plt.imshow(train_images[i])
    plt.tight_layout() 

In [ ]:
y=np.array([1 if value=='Pnemonia' else 0 for value in df_train_balanced['class']])

In [ ]:
y_valid=np.array([1 if value=='Pneumonia' else 0 for value in df_test_balanced['class']])

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
install('image-quality')


### Blind/Referenceless Image Spatial Quality Evaluator (BRISQUE)

The Blind/Referenceless Image Spatial Quality Evaluator (BRISQUE)  algorithm measures the quality score of an image with computational performance.   The algorithm  extracts the pointwise numbers of sectionally normalized luminance signs, and measures image naturalness based on measured deviations from a natural picture model.  The algorithm model the distribution of pairwise statistics of neighboring normalized luminance signals, which provide distortion orientation info.  Though multiscale, the version employs simple to calculate features making it computationally fast and time-efficient

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error
from scipy import signal
import math
import imquality.brisque as brisque
from tqdm import tqdm

best_score=10000000
best_a=-1
best_b=-1
best_c=-1
best_d=-1
best_0=-1
best_1=-1
plt.figure(figsize=(10,8))
plt.subplots_adjust(left=0.1, right=0.2, top=0.2, bottom=0.1)

for a in tqdm(range(0,6)):
    for b in range(0,6):
  
      
        
        mfx = fuzz.trapmf(train_images[2].flatten(), [a/10, a/10+b/10,200,200])


        

        hist=np.histogram(mfx.reshape(256,256,3))
       
        if ((hist[0][0]>0) & (hist[0][-1]>0)):
            score= brisque.score(mfx.reshape(256,256,3))
            plt.subplot(3, 3, a + 1,aspect = 'equal')
            plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
            plt.title('Brisque score '+str(score))
            plt.margins(0, 0)

            plt.imshow(mfx.reshape(256,256,3))
            plt.axis('off')
           
         
            if score<best_score:
                best_score=score
                best_a=a/10
                best_b=a/10+0.2
                best_c=200
                best_d=200
                best_0=hist[0][0]
                best_1=hist[0][-1]



        
print('best parameters ',best_a,best_b,best_c,best_d,best_score,best_0,best_1)
plt.tight_layout(pad=0.0) 

In [ ]:
images_fuzzy=[]
for image,name in zip(train_images,filenames):
            
            fuzzy_image=fuzzy(image,best_a,best_a+best_b,best_c,best_d)
           
            
            images_fuzzy.append(fuzzy_image)
        


images_valid_fuzzy=[]
for image in valid_images:
    
            fuzzy_image=fuzzy(image,best_a,best_a+best_b,best_c,best_d)
       
           
    
    

            images_valid_fuzzy.append(fuzzy_image)


In [ ]:
images_seg=[]
for image,name in zip(train_images,filenames):
            
            seg_image=seg_(image)
           
            
            images_seg.append(seg_image)
        


images_valid_seg=[]
for image in valid_images:
    
            seg_image=seg_(image)
       
           
    
    

            images_valid_seg.append(seg_image)


### Image Segmentation using K-Means

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
for i in range(9):
    
    plt.subplot(3, 3, i + 1,aspect = 'equal')
    plt.imshow(images_seg[i])
    plt.tight_layout() 

### Sample of X-Ray with Fuzzy

In [ ]:
df_train_file=pd.DataFrame(columns=df_train_balanced.columns)
for file in train_no_fuzzy.filenames:
 
    df_train_aux=df_train[df_train['filename']==file]
    df_train_file=pd.concat([df_train_file,df_train_aux])
    
df_valid_file=pd.DataFrame(columns=df_valid_balanced.columns)
for file in valid_no_fuzzy.filenames:
 
    df_valid_aux=df_train[df_train['filename']==file]
    df_valid_file=pd.concat([df_valid_file,df_valid_aux])

In [ ]:
df_train_file

In [ ]:
df_train_file_aux=df_train_file.reset_index()

In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(figsize=(20,8))
f.add_subplot(1,2, 1)


index=df_train_file_aux.index[df_train_file_aux['Label']=='Normal'].to_list()[1]
mfx = fuzz.trapmf(train_images[2].flatten(), [best_a,best_a+best_b,best_c,best_d])


raw_image = images_fuzzy[index]
plt.imshow(raw_image)
plt.colorbar()
plt.title('Image Normal with best fuzzy parameters')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()


In [ ]:

import matplotlib.pyplot as plt
print('Display Fuzzy Filter 2 Images')

# Adjust the size of your images
plt.figure(figsize=(10,8))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img =images_fuzzy[i]
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
# Adjust subplot parameters to give specified padding
plt.tight_layout()  

In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(figsize=(20,8))
f.add_subplot(1,2, 1)


index=df_train_file_aux.index[df_train_file_aux['Label']!='Normal'].to_list()[1]



raw_image = images_fuzzy[index]
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('COVID-19 Image Fuzzy Filter')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()


In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(figsize=(20,8))
f.add_subplot(1,2, 1)


index=df_train_file_aux.index[df_train_file_aux['Label']=='Normal'].to_list()[1]



raw_image = images_fuzzy[index]
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Normal Image Fuzzy Filter')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()


In [ ]:

train_df=df_train_file.drop(['Unnamed: 0','Dataset_type','class'],axis=1)
valid_df=df_valid_file.drop(['Unnamed: 0','Dataset_type','class'],axis=1)



In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_=le.fit_transform(y)
y_valid_=le.transform(y_valid)

### Method to create Single Input Model

In [ ]:


import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121

import sys
import subprocess
import pkg_resources
required = {'efficientnet'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    !pip install -U efficientnet
import efficientnet.keras as efn 

from efficientnet.keras import EfficientNetB0 as Net
from efficientnet.keras import EfficientNetB3 
def create_single_model(weights,dropout_var,network):

    input_shape=(256,256,3)
        
    model1 = VGG16(include_top=False, input_shape=(256, 256, 3))
    if network[0]=='VGG16':
        model1 = VGG16(include_top=False, input_shape=(256, 256, 3))
        
    if network[0]=='ResNet152V2':
        model1 = ResNet152V2(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[0]=='InceptionV3':
        model1 = InceptionV3(include_top=False, input_shape=(256, 256, 3))
        
    if network[0]=='Xception':
        model1 = Xception(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[0]=='EfficientNetB3':
        model1 = EfficientNetB3(weights="imagenet", include_top=False, input_shape=input_shape)




    
    
    
    flat1_ = Flatten() (model1.output)
    dropout_ = Dropout(dropout_var)(flat1_)
    dense1_ = Dense(weights, activation='relu')(dropout_)
    batch_ =  BatchNormalization()(dense1_)
    dense2_ = Dense(weights, activation='relu')(batch_)
    dropout_ = Dropout(dropout_var)(dense2_)
    
    dense3_ = Dense(4, activation="relu")(dropout_ )
    output = Dense(1, activation="sigmoid")(dense3_)

    model= Model(inputs=model1.input, outputs=output)

    



    return model

### Callbacks for Training

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

check=ModelCheckpoint(filepath='/kaggle/working/modelvgg.h5',monitor='val_acc',save_best_only=True)
callbacks=[reduce_lr,check]




### Train with K-Means clustering

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']

# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
   

#                 print('Testing with  {} and {}  '.format(network,weights))
                
#                 print('Creating model')
#                 model=create_single_model(weights,0.5,[network])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[np.array(images_seg)], y=y,
#                     validation_data=([valid_images],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/seg_nofuzzy_History to {}  with weight of  {} '.format(network,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/seg_nofuzzy_Loss to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/seg_nofuzzy_AUC to {}  with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} with weight of  {} '.format(network,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/seg_nofuzzy_Precision to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 print('Recall to {}  with weight of  {} '.format(network,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/seg_nofuzzy_Recall to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()

    
    
 

### Train single input model without Segmentation

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']

# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
   

#                 print('Testing with  {} and {}  '.format(network,weights))
                
#                 print('Creating model')
#                 model=create_single_model(weights,0.5,[network])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[train_images], y=y,
#                     validation_data=([valid_images],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/single_nofuzzy_History to {}  with weight of  {} '.format(network,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_nofuzzy_Loss to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_nofuzzy_AUC to {}  with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} with weight of  {} '.format(network,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_nofuzzy_Precision to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 print('Recall to {}  with weight of  {} '.format(network,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_nofuzzy_Recall to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()

    
    
 

### Train single input model with Fuzzy Filter Images

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']

# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
   

#                 print('Testing with  {} and {}  '.format(network,weights))
                
#                 print('Creating model')
#                 model=create_single_model(weights,0.5,[network])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[np.array(images_fuzzy)], y=y,
#                     validation_data=([valid_images],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/single_fuzzyfuzzy_History to {}  with weight of  {} '.format(network,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_fuzzyfuzzy_Loss to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_fuzzyfuzzy_AUC to {}  with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {}  with weight of  {} '.format(network,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} with weight of  {} '.format(network,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_fuzzyfuzzy_Precision to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()


#                 print('Recall to {}  with weight of  {} '.format(network,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {}  with weight of  {} '.format(network,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/single_fuzzyfuzzy_Recall to {} with weight of  {} '.format(network,weights)+'.png')
#                 plt.show()

    
    
 

### Method to create model

In [ ]:


import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121

import sys
import subprocess
import pkg_resources
required = {'efficientnet'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    !pip install -U efficientnet
import efficientnet.keras as efn 

from efficientnet.keras import EfficientNetB0 as Net
from efficientnet.keras import EfficientNetB3 
def create_model(weights,dropout_var,network):

    input_shape=(256,256,3)
        
    model1 = VGG16(include_top=False, input_shape=(256, 256, 3))
    if network[0]=='VGG16':
        model1 = VGG16(include_top=False, input_shape=(256, 256, 3))
        
    if network[0]=='ResNet152V2':
        model1 = ResNet152V2(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[0]=='InceptionV3':
        model1 = InceptionV3(include_top=False, input_shape=(256, 256, 3))
        
    if network[0]=='Xception':
        model1 = Xception(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[0]=='EfficientNetB3':
        model1 = EfficientNetB3(weights="imagenet", include_top=False, input_shape=input_shape)



    flat1 = Flatten() (model1.output)
    dropout1 = Dropout(dropout_var)(flat1)
    dense1 = Dense(weights, activation='relu')(dropout1)
    batch1 =  BatchNormalization()(dense1)
    dense12 = Dense(weights, activation='relu')(batch1)
    dropout12 = Dropout(dropout_var)(dense12)

    model1 = Model(inputs=model1.input, outputs=dropout12)


    import sys
    import subprocess
    import pkg_resources

    required = {'efficientnet'}
    installed = {pkg.key for pkg in pkg_resources.working_set}
    missing = required - installed



    from efficientnet.keras import EfficientNetB0 as Net
    from efficientnet.keras import EfficientNetB3 as Net7

    input_shape=(256,256,3)

    model2 = Net7(weights="imagenet", include_top=False, input_shape=input_shape)
    
    
    if network[1]=='VGG16':
        model2 = VGG16(include_top=False, input_shape=(256, 256, 3))
        
    if network[1]=='ResNet152V2':
        model2 = ResNet152V2(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[1]=='InceptionV3':
        model2 = InceptionV3(include_top=False, input_shape=(256, 256, 3))
        
    if network[1]=='Xception':
        model2 = Xception(include_top=False, input_shape=(256, 256, 3))
        
        
    if network[1]=='EfficientNetB3':
        model2 = EfficientNetB3(weights="imagenet", include_top=False, input_shape=input_shape)



    flat1_ = Flatten() (model2.output)
    dropout_ = Dropout(dropout_var)(flat1_)
    dense1_ = Dense(weights, activation='relu')(dropout_)
    batch_ =  BatchNormalization()(dense1_)
    dense2_ = Dense(weights, activation='relu')(batch_)
    dropout_ = Dropout(dropout_var)(dense2_)
    model2= Model(inputs=model2.input, outputs=dropout_)


    modeld = Sequential()
    modeld.add(Dense(14, input_dim=4, activation="relu"))
    modeld.add(Dense(28,  activation="relu"))
    modeld.add(Dropout(dropout_var))
    modeld.add(Dense(4, activation="relu"))



    model2.trainable = False
    combinedInput = concatenate([model1.output,model2.output])

    x = Dense(4, activation="relu")(combinedInput)
    x = Dense(1, activation="sigmoid")(x)

    modelall = Model(inputs=[model1.input,model2.input], outputs=x)





    return modelall

In [ ]:
model=create_model(20,0.5,["VGG16","InceptionV3"])

tf.keras.utils.plot_model(
     model,
     to_file="/kaggle/working/model.png",
)

### Callbacks

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2)

check=ModelCheckpoint(filepath='/kaggle/working/modelvgg.h5',monitor='val_acc',save_best_only=True)
callbacks=[reduce_lr,check]




In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Reports of comparative experiments

## Read reports with KMeans Segmentation

In [ ]:
def select_networks_rows(experiments,network1,fuzzy):
    
    return experiments[(experiments[network1]==1) & (experiments[fuzzy]==1)]
    

In [ ]:
def get_point(dataframe):
    best_point=-1
    best_accuracy=-1
    best_auc=-1
    best_precision=-1
    best_recall=-1
    for i,r in dataframe.iterrows():
        local_point=r['VAL Accuracy']+r['VAL AUC']+r['VAL Precision']+r['VAL Recall']
        if local_point>best_point:
            best_point=local_point
           
            best_accuracy=r['VAL Accuracy']
            best_auc=r['VAL AUC']
            best_precision=r['VAL Precision']
            best_recall=r['VAL Recall']
            
    return best_accuracy,best_auc,best_precision,best_recall

        
    

In [ ]:
import pandas as pd

networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']

best_auc=-10

all_data=[]
experiments=pd.DataFrame(columns=['Loss','VAL AUC','VAL Precision','VAL Recall','VAL Accuracy'])

weights=20


import os,glob
os.chdir('/kaggle/input/papercovidseg/')
files=glob.glob('*.csv')




for file in files:
    
    
                
            
                    df_h=pd.DataFrame()
                    try:
                        df_h=pd.read_csv(file)
                    except:
                        pass
                    VGG16=0
                    ResNet152V2=0
                    InceptionV3=0
                    EfficientNetB3=0
                    seg=0
                    fuzzy=0
                    nofuzzy=0
                    if len(df_h)>0:
                        
                        if 'VGG16' in file:
                            VGG16=1
                        if 'ResNet152V2' in file:
                            ResNet152V2=1
                        if 'InceptionV3' in file:
                            InceptionV3=1
                        if 'EfficientNetB3' in file:
                            EfficientNetB3=1
                            
                       
                        if 'seg' in file:
                            seg=1
                            
                        #print(df_h.columns)
                        for i,r in df_h.iterrows():
                            loss=r['loss']
                            val_auc=r['val_auc']
                            val_precision=r['val_precision_2']
                            val_recall=r['val_recall_2']
                            val_accuracy=r['val_accuracy']
                            time=r['Unnamed: 0']
                            lr=r['lr']

                            experiments=experiments.append({'VGG16':VGG16,'ResNet152V2':ResNet152V2,
                                                            'InceptionV3':InceptionV3,'EfficientNetB3':EfficientNetB3
                                                            ,'Loss':loss,'VAL AUC':val_auc,
                                                           'VAL Precision':val_precision,'VAL Recall':val_recall,
                                                           'VAL Accuracy':val_accuracy,
                                                           'time':time,'lr':lr,'Fuzzy':fuzzy,'NoFuzzy':nofuzzy,'SEG':seg},ignore_index=True)
                        
               # except:
                #    pass
    
 

### Results with KMeans segmentation

In [ ]:
networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

results=pd.DataFrame()
    
    
    
for network in networks:
        
      
            
            
            selected=select_networks_rows(experiments,network,'SEG')
            best_accuracy,best_auc,best_precision,best_recall=get_point(selected)
            results=results.append({'Model1':network,'Accuracy':best_accuracy,'AUC':best_auc,
                            'Precision':best_precision,'Recall':best_recall},ignore_index=True)
            

results_kmeans=results
results
        

### Read reports with train images and Single Input Model

In [ ]:
import pandas as pd

networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']

best_auc=-10

all_data=[]
experiments=pd.DataFrame(columns=['Loss','VAL AUC','VAL Precision','VAL Recall','VAL Accuracy'])

weights=20


import os,glob
os.chdir('/kaggle/input/papercovidsingle/')
files=glob.glob('*.csv')




for file in files:
    
    
                
            
                    df_h=pd.DataFrame()
                    try:
                        df_h=pd.read_csv(file)
                    except:
                        pass
                    VGG16=0
                    ResNet152V2=0
                    InceptionV3=0
                    EfficientNetB3=0
                    fuzzy=0
                    nofuzzy=0
                    if len(df_h)>0:
                        
                        if 'VGG16' in file:
                            VGG16=1
                        if 'ResNet152V2' in file:
                            ResNet152V2=1
                        if 'InceptionV3' in file:
                            InceptionV3=1
                        if 'EfficientNetB3' in file:
                            EfficientNetB3=1
                            
                        if 'nofuzzy' in  file:
                            nofuzzy=1
                        
                        if 'fuzzyfuzzy' in file:
                            fuzzy=1
                            
                        #print(df_h.columns)
                        for i,r in df_h.iterrows():
                            loss=r['loss']
                            val_auc=r['val_auc']
                            val_precision=r['val_precision_2']
                            val_recall=r['val_recall_2']
                            val_accuracy=r['val_accuracy']
                            time=r['Unnamed: 0']
                            lr=r['lr']

                            experiments=experiments.append({'VGG16':VGG16,'ResNet152V2':ResNet152V2,
                                                            'InceptionV3':InceptionV3,'EfficientNetB3':EfficientNetB3
                                                            ,'Loss':loss,'VAL AUC':val_auc,
                                                           'VAL Precision':val_precision,'VAL Recall':val_recall,
                                                           'VAL Accuracy':val_accuracy,
                                                           'time':time,'lr':lr,'Fuzzy':fuzzy,'NoFuzzy':nofuzzy},ignore_index=True)
                        
               # except:
                #    pass
    
 

In [ ]:
networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

results=pd.DataFrame()
    
    
    
for network in networks:
        
      
            
            
            selected=select_networks_rows(experiments,network,'NoFuzzy')
            best_accuracy,best_auc,best_precision,best_recall=get_point(selected)
            results=results.append({'Model1':network,'Accuracy':best_accuracy,'AUC':best_auc,
                            'Precision':best_precision,'Recall':best_recall},ignore_index=True)
            

results_no_fuzzy_single=results
results
        

In [ ]:
results_no_fuzzy_single[results_no_fuzzy_single['Accuracy']>results_kmeans['Accuracy']]

In [ ]:
networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

results=pd.DataFrame()
    
    
    
for network in networks:
        
      
            
            
            selected=select_networks_rows(experiments,network,'Fuzzy')
            best_accuracy,best_auc,best_precision,best_recall=get_point(selected)
            results=results.append({'Model1':network,'Accuracy':best_accuracy,'AUC':best_auc,
                            'Precision':best_precision,'Recall':best_recall},ignore_index=True)
            

results_fuzzy_single=results
results
        

In [ ]:
df1=select_networks_rows(experiments,"ResNet152V2",'Fuzzy')
df2=select_networks_rows(experiments,"ResNet152V2",'NoFuzzy')

df3=select_networks_rows(experiments,"InceptionV3",'Fuzzy')
df4=select_networks_rows(experiments,"InceptionV3",'NoFuzzy')

import altair as alt

df_=pd.DataFrame()

for i,r in df1.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'Fuzzy ResNet152V2'},ignore_index=True)

for i,r in df2.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'NoFuzzy ResNet152V2'},ignore_index=True)

for i,r in df3.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'Fuzzy InceptionV3'},ignore_index=True)

for i,r in df4.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'NoFuzzy InceptionV3'},ignore_index=True)
                         
                         
                         
alt.Chart(df_).mark_line().encode(
x=alt.X('x',title='epoch'),
y=alt.Y('y',title='AUC'),
color='class')

In [ ]:
df1=select_networks_rows(experiments,"ResNet152V2",'Fuzzy')
df2=select_networks_rows(experiments,"ResNet152V2",'NoFuzzy')

df3=select_networks_rows(experiments,"InceptionV3",'Fuzzy')
df4=select_networks_rows(experiments,"InceptionV3",'NoFuzzy')

import altair as alt

df_=pd.DataFrame()

for i,r in df1.iterrows():
    df_=df_.append({'x':r['time'],'y':r['Loss'],'class':'Fuzzy ResNet152V2'},ignore_index=True)

for i,r in df2.iterrows():
    df_=df_.append({'x':r['time'],'y':r['Loss'],'class':'NoFuzzy ResNet152V2'},ignore_index=True)

for i,r in df3.iterrows():
    df_=df_.append({'x':r['time'],'y':r['Loss'],'class':'Fuzzy InceptionV3'},ignore_index=True)

for i,r in df4.iterrows():
    df_=df_.append({'x':r['time'],'y':r['Loss'],'class':'NoFuzzy InceptionV3'},ignore_index=True)
                         
                         
                         
alt.Chart(df_).mark_line().encode(
x=alt.X('x',title='epoch'),
y=alt.Y('y',title='Loss'),
color='class')

In [ ]:
def compare(metric,results_fuzzy,results_no_fuzzy):
    #results_fuzzy=results_fuzzy.filter([metric])
    #results_no_fuzzy=results_no_fuzzy.filter([metric])
    r1=results_fuzzy[results_fuzzy[metric]>results_no_fuzzy[metric]]
    r2=results_no_fuzzy[results_no_fuzzy[metric]<results_fuzzy[metric]]
    r3=pd.concat([r1,r2],axis=1)
    return r3

In [ ]:
r3=compare('AUC',results_fuzzy_single,results_no_fuzzy_single)
r3

In [ ]:
import pandas as pd

networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

all_data=[]
experiments=pd.DataFrame(columns=['Loss','VAL AUC','VAL Precision','VAL Recall','VAL Accuracy'])

weights=20


import os,glob
os.chdir('/kaggle/input/paper-covid-6-7/')
files=glob.glob('*.csv')




for file in files:
    
    
                
            
                    df_h=pd.DataFrame()
                    try:
                        df_h=pd.read_csv(file)
                    except:
                        pass
                    VGG16=0
                    ResNet152V2=0
                    InceptionV3=0
                    EfficientNetB3=0
                    fuzzy=0
                    nofuzzy=0
                    if len(df_h)>0:
                        
                        if 'VGG16' in file:
                            VGG16=1
                        if 'ResNet152V2' in file:
                            ResNet152V2=1
                        if 'InceptionV3' in file:
                            InceptionV3=1
                        if 'EfficientNetB3' in file:
                            EfficientNetB3=1
                            
                        if 'nofuzzy' in  file:
                            nofuzzy=1
                        
                        if 'fuzzyfuzzy' in file:
                            fuzzy=1
                            
                        #print(df_h.columns)
                        for i,r in df_h.iterrows():
                            loss=r['loss']
                            val_auc=r['val_auc']
                            val_precision=r['val_precision_2']
                            val_recall=r['val_recall_2']
                            val_accuracy=r['val_accuracy']
                            time=r['Unnamed: 0']
                            lr=r['lr']

                            experiments=experiments.append({'VGG16':VGG16,'ResNet152V2':ResNet152V2,
                                                            'InceptionV3':InceptionV3,'EfficientNetB3':EfficientNetB3
                                                            ,'Loss':loss,'VAL AUC':val_auc,
                                                           'VAL Precision':val_precision,'VAL Recall':val_recall,
                                                           'VAL Accuracy':val_accuracy,
                                                           'time':time,'lr':lr,'Fuzzy':fuzzy,'NoFuzzy':nofuzzy},ignore_index=True)
                        
               # except:
                #    pass
    
 

In [ ]:
experiments

In [ ]:
#vgg16nof=experiments[(experiments['VGG16']==1) & (experiments['NoFuzzy']==1)]

In [ ]:
def select_networks_rows(experiments,network1,network2,fuzzy):
    
    return experiments[(experiments[network1]==1) & (experiments[network2]==1) & (experiments[fuzzy]==1)]
    

## MultiInput results without fuzzy

In [ ]:
networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

results=pd.DataFrame()
    
    
    
for network in networks:
        
        for network2 in networks2:
            
            if network!=network2:
            

                selected=select_networks_rows(experiments,network,network2,'NoFuzzy')
                if len(selected)>0:
                    best_accuracy,best_auc,best_precision,best_recall=get_point(selected)
                    results=results.append({'Model1':network,'Model2':network2,'Accuracy':best_accuracy,'AUC':best_auc,
                                    'Precision':best_precision,'Recall':best_recall},ignore_index=True)


results_no_fuzzy=results
results
        

### MultiInput results with Fuzzy

In [ ]:
networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
best_auc=-10

results=pd.DataFrame()
    
    
    
for network in networks:
        
        for network2 in networks2:
            
            if network!=network2:
            

                selected=select_networks_rows(experiments,network,network2,'Fuzzy')
                best_accuracy,best_auc,best_precision,best_recall=get_point(selected)
                results=results.append({'Model1':network,'Model2':network2,'Accuracy':best_accuracy,'AUC':best_auc,
                                'Precision':best_precision,'Recall':best_recall},ignore_index=True)

results_fuzzy=results
results
        

In [ ]:
results_fuzzy['AUC']=round(results_fuzzy['AUC'], 2)

## Networks results

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.parallel_categories(results_fuzzy, dimensions=['Model1', 'Model2', 'AUC'],
                color="Accuracy", color_continuous_scale=px.colors.sequential.Inferno,
                )
fig.show()

### Comparation between Fuzzy and no Fuzzy MultiInput results

In [ ]:
r3=compare('AUC',results_fuzzy,results_no_fuzzy)
r3

In [ ]:
two_networks_fuzzy=results_fuzzy.groupby(['Model1']).mean()
two_networks_fuzzy=two_networks_fuzzy.reset_index()
two_networks_fuzzy


In [ ]:
results_fuzzy_single

## Comparation between Single and MultiInput

In [ ]:
dataframe=pd.DataFrame()
metric='Accuracy'
for n in two_networks_fuzzy['Model1'].unique():
    value=two_networks_fuzzy[two_networks_fuzzy['Model1']==n][metric].values[0]
    value1=results_fuzzy_single[results_fuzzy_single['Model1']==n][metric].values[0]
    if value>value1:
        dataframe=pd.concat([dataframe,two_networks_fuzzy[two_networks_fuzzy['Model1']==n]],axis=0)
dataframe

In [ ]:
results_fuzzy[results_fuzzy['AUC']>results_no_fuzzy['AUC']]

In [ ]:
results_fuzzy[results_fuzzy['Recall']>results_no_fuzzy['Recall']]

In [ ]:
df1=select_networks_rows(experiments,"VGG16","InceptionV3",'Fuzzy')

df2=select_networks_rows(experiments,"VGG16","InceptionV3",'NoFuzzy')

df3=select_networks_rows(experiments,"ResNet152V2","EfficientNetB3",'Fuzzy')
df4=select_networks_rows(experiments,"ResNet152V2","EfficientNetB3",'NoFuzzy')

import altair as alt

df_=pd.DataFrame()

#for i,r in df1.iterrows():
#    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'Fuzzy VGG16 Inception V3'},ignore_index=True)

#for i,r in df2.iterrows():
#    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'NoFuzzy ResNet152V2'},ignore_index=True)

for i,r in df3.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'Fuzzy InceptionV3'},ignore_index=True)

for i,r in df4.iterrows():
    df_=df_.append({'x':r['time'],'y':r['VAL AUC'],'class':'NoFuzzy InceptionV3'},ignore_index=True)
                         
                         
                         
alt.Chart(df_).mark_line().encode(
x=alt.X('x',title='epoch'),
y=alt.Y('y',title='AUC'),
color='class')

In [ ]:
train_images.shape

## Code to run experiments

## Train with fuzzy images

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
#         for network2 in networks2:
        
#             if network!=network2:
#                 print('Testing with  {} and {} and {} '.format(network,weights,network2))
                
#                 print('Creating model')
#                 model=create_model(weights,0.5,[network,network2])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[train_images,np.array(images_fuzzy)], y=y,
#                     validation_data=([valid_images,np.array(images_valid_fuzzy)],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/History to {} and {} with weight of  {} '.format(network,network2,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/Loss to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/AUC to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/Precision to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 print('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/Recall to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()

    
    
 

## Train with no-fuzzy images

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
#         for network2 in networks2:
        
#             if network!=network2:
#                 print('Testing with  {} and {} and {} '.format(network,weights,network2))
                
#                 print('Creating model')
#                 model=create_model(weights,0.5,[network,network2])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[train_images,train_images], y=y,
#                     validation_data=([valid_images,valid_images],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/nofuzzyHistory to {} and {} with weight of  {} '.format(network,network2,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/nofuzzyLoss to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/nofuzzyAUC to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/nofuzzyPrecision to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 print('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/nofuzzyRecall to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()

    
    
 

## Experiment with fuzzy images a

In [ ]:
# networks=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# networks2=['VGG16','ResNet152V2','InceptionV3','EfficientNetB3']
# best_auc=-10

# for weights in range(20,22,20):
    
    
    
#     for network in networks:
        
#         for network2 in networks2:
        
#             if network!=network2:
#                 print('Testing with  {} and {} and {} '.format(network,weights,network2))
                
#                 print('Creating model')
#                 model=create_model(weights,0.5,[network,network2])
#                 opt = Adam(lr=1e-3, decay=1e-3 / 100)
#                 print('Model created')
#                 losses = tf.keras.losses.BinaryCrossentropy()
#                 model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                                  tf.keras.metrics.Recall(name='recall_2'),tf.keras.metrics.AUC(name='auc'),'accuracy'])
#                 print('Model compiled')
#                 history=model.fit(
#                     x=[train_images,np.array(images_fuzzy)], y=y,
#                     validation_data=([valid_images,valid_images],y_valid),
#                     epochs=10, batch_size=10,callbacks=callbacks)
#                 auc=max(history.history['val_auc'])
#                 if auc>best_auc:
#                     best_auc=auc
#                     best_network=network
#                     best_weight=weights

#                 print('val_auc',auc)
#                 if (auc)>0.98:
#                     print('find')

#                 df_history=pd.DataFrame.from_dict(history.history)
#                 df_history.to_csv('/kaggle/working/fuzzyfuzzyHistory to {} and {} with weight of  {} '.format(network,network2,weights)+'.csv')

#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['loss'])
#                 plt.plot(history.history['val_loss'])
#                 plt.title('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('loss')
#                 plt.xlabel('epoch')
#                 plt.legend(['loss', 'val_loss'], loc='upper left')
#                 plt.savefig('/kaggle/working/fuzzyfuzzyLoss to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['auc'])
#                 plt.plot(history.history['val_auc'])
#                 plt.title('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('AUC')
#                 plt.xlabel('epoch')
#                 plt.legend(['AUC', 'val AUC'], loc='upper left')
#                 plt.savefig('/kaggle/working/fuzzyfuzzyAUC to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 print('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.plot(history.history['precision_2'])
#                 plt.plot(history.history['val_precision_2'])
#                 plt.title('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('precision')
#                 plt.xlabel('epoch')
#                 plt.legend(['precision', 'val_precision'], loc='upper left')
#                 plt.savefig('/kaggle/working/fuzzyfuzzyPrecision to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()


#                 print('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 from matplotlib.pyplot import figure
#                 figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
#                 plt.plot(history.history['recall_2'])
#                 plt.plot(history.history['val_recall_2'])
#                 plt.title('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
#                 plt.ylabel('Recall')
#                 plt.xlabel('epoch')
#                 plt.legend(['recall', 'val_recall'], loc='upper left')
#                 plt.savefig('/kaggle/working/fuzzyfuzzyRecall to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
#                 plt.show()

    
    
 

## Experiment with best model found

In [ ]:

# best_auc=-10


# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.callbacks import ModelCheckpoint


# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=0)

# check=ModelCheckpoint(filepath='/kaggle/working/modelvgg.h5',monitor='val_acc',save_best_only=True)
# callbacks=[reduce_lr,check,tf.keras.callbacks.EarlyStopping(
#     monitor='val_acc', min_delta=1, patience=3, verbose=1, mode='max',
#     baseline=None, restore_best_weights=True
# )]


# weights=20
# network='ResNet152V2'
# network2='VGG16'

# print('Testing with  {} and {} and {} '.format(network,weights,network2))

# print('Creating model')
# model=create_model(weights,0.5,[network,network2])
# opt = Adam(lr=1e-3, decay=1e-3 / 100)
# print('Model created')
# losses = tf.keras.losses.BinaryCrossentropy()
# model.compile(loss="binary_crossentropy" ,optimizer=opt,metrics=[tf.keras.metrics.Precision(name='precision_2'),
#                                                                  tf.keras.metrics.Recall(name='recall_2'),
#                                                                  tf.keras.metrics.AUC(name='auc'),'accuracy'])
# print('Model compiled')
# history=model.fit(
#     x=[train_images,np.array(images_fuzzy)], y=y,
#     validation_data=([valid_images,valid_images],y_valid),
#     epochs=100, batch_size=10,callbacks=callbacks)
# auc=max(history.history['val_auc'])
# if auc>best_auc:
#     best_auc=auc
#     best_network=network
#     best_weight=weights

# print('val_auc',auc)
# if (auc)>0.98:
#     print('find')

# df_history=pd.DataFrame.from_dict(history.history)
# df_history.to_csv('/kaggle/working/bestHistory to {} and {} with weight of  {} '.format(network,network2,weights)+'.csv')

# from matplotlib.pyplot import figure
# figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
# print('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Loss to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'val_loss'], loc='upper left')
# plt.savefig('/kaggle/working/bestLoss to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
# plt.show()


# from matplotlib.pyplot import figure
# figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
# print('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.plot(history.history['auc'])
# plt.plot(history.history['val_auc'])
# plt.title('AUC to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.ylabel('AUC')
# plt.xlabel('epoch')
# plt.legend(['AUC', 'val AUC'], loc='upper left')
# plt.savefig('/kaggle/working/fbestAUC to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
# plt.show()


# from matplotlib.pyplot import figure
# figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
# print('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.plot(history.history['precision_2'])
# plt.plot(history.history['val_precision_2'])
# plt.title('Precision to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.ylabel('precision')
# plt.xlabel('epoch')
# plt.legend(['precision', 'val_precision'], loc='upper left')
# plt.savefig('/kaggle/working/bestPrecision to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
# plt.show()


# print('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
# from matplotlib.pyplot import figure
# figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
# plt.plot(history.history['recall_2'])
# plt.plot(history.history['val_recall_2'])
# plt.title('Recall to {} and {} with weight of  {} '.format(network,network2,weights))
# plt.ylabel('Recall')
# plt.xlabel('epoch')
# plt.legend(['recall', 'val_recall'], loc='upper left')
# plt.savefig('/kaggle/working/bestRecall to {} and {} with weight of  {} '.format(network,network2,weights)+'.png')
# plt.show()





## Use trained model

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121

weights=20
network='ResNet152V2'
network2='VGG16'



model=create_model(weights,0.5,[network,network2])

In [ ]:

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy



#json_file = open('/kaggle/input/modelcovid1/model1.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#lmodel = model_from_json(loaded_model_json)



model.load_weights('/kaggle/input/modelcovid1/covid1.h5')

In [ ]:
model.save_weights('/kaggle/working/covid.h5')

### Make predictions

In [ ]:
predictions=model.predict([valid_images,valid_images])

### Apply threshold on predictions

In [ ]:
ypred=[1 if x>0.5 else 0 for x in predictions]

In [ ]:
len(valid_images)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_valid,ypred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix


In [ ]:
model_json = model.to_json()
with open("/kaggle/working/model.json", "w") as json_file:
    json_file.write(model_json)

### Confusion Matrix

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_valid,ypred)

In [ ]:
plot_confusion_matrix(cm           = c, 
                      normalize    = False,
                      target_names = ['Normal', 'SARS-COVID-19'],
                      title        = "Confusion Matrix")

### ROC curve

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset

# predict probabilities
lr_probs = predictions
# keep probabilities for the positive outcome only
lr_probs = lr_probs
# calculate scores
ns_auc = roc_auc_score(y_valid, ypred)
lr_auc = roc_auc_score(y_valid, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_valid, ypred)
lr_fpr, lr_tpr, _ = roc_curve(y_valid, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y_valid,predictions)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_valid.ravel(), predictions.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure(figsize=(10,10))
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


### Machine Learning Explainable

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import tensorflow.keras.backend as K

def gradCAM(img, intensity=0.7, res=256):
  
  x = img
 
  x = np.expand_dims(x, axis=0)
  #x = preprocess_input(x)

  preds = model.predict([[x],[x]])
  #print(decode_predictions(preds)[0][0][1]) # prints the class of image

  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer('block5_conv3')
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate([[x],[x]])
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((16, 16))
    

  #img =valid_images[4]*255




  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
  # We convert the heatmap to RGB
  heatmap = np.uint8(255 * heatmap)
    
  # We apply the heatmap to the original image
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

  # 0.4 here is a heatmap intensity factor
  s_img = heatmap * 0.4 + img


  img1 = heatmap * intensity + img
  plt.figure(figsize=(10,10))

  i=0
  plt.subplot(3, 3, i + 1,aspect = 'equal') 
  plt.imshow(cv2.resize(img, (res, res)))

  i=i+1 
  plt.subplot(3, 3, i + 1,aspect = 'equal') 
    
  plt.imshow(heatmap)
 
  i=i+1 
  plt.subplot(3, 3, i + 1,aspect = 'equal') 
  
  plt.imshow(cv2.resize(s_img, (res, res)))
 

  
  
  plt.tight_layout() 
  plt.show()
 



    


In [ ]:


for i in range(10):
    print(y_valid[i])
    gradCAM(valid_images[i])


In [ ]:
for i in range(10):
    print(y_valid[i])
    gradCAM(images_valid_fuzzy[i])